In [16]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset. I can't get my wget to work so I simply went to the URL and downloaded the file and placed
# it in the correct folder. It's about 1 MB.
# https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


In [17]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [18]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [19]:
# Here are the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [20]:
# Here are all the unique characters in the data set. Lower and upper case, periods, exclamations points, question marks, commas, etc. 65 unique characters.
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [21]:
# Tokenizer
# create the map from characters to integers. This is basically all the same from the previous 'makemore' build
# This is a very simple codebook, a very small vocabulary. This is nice, but the tradeoff is that we have longer lists of integers
stoi = { ch:i for i,ch in enumerate(chars) } # enumerate returns an iterator for each character in chars. It returns a tuple of (index, character). Ex (0, 'a'), (1, 'b'), etc. The code wrapped around that creates a dictionary.
itos = { i:ch for i,ch in enumerate(chars) } # Same thing here, but we are going from integers back to characters.
encode = lambda s: [stoi[c] for c in s] # the encoder takes a string and outputs a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # the decoder takes a list of integers and outputs a string

print(encode("hii world"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 61, 53, 56, 50, 42]
hii there


In [22]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earlier will look like this to the GPT

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [23]:
#creating the training and val sets
n = int(0.9*len(data)) #first 90% is training data, the rest is val
train_data = data[:n] # training set
val_data = data[n:] # validation set. Don't use this for training, only to check the model's loss after its been trained

In [24]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [25]:
x = train_data[:block_size] #input data. This is what the model will train on
y = train_data[1:block_size + 1] #desired output, or target, when the input is x
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [26]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train') # These are the actual batch inputs (xb) and targets (yb) that the model will train on. In this batch, there are 32 unique training inputs, outlined in english below
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [27]:
print(xb) #our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    
    
# B: Batch size, the number of sequences processed in parallel. In deep learning, processing
# multiple sequences in parallel (batch processing) is more efficient than processing them one by one. 
# The batch size determines how many sequences are processed simultaneously.
# If B = 4, it means that 4 sequences are being processed in parallel.

# T: Sequence length, the number of tokens in the input sequence.In language models, sequences are often 
# broken down into smaller chunks or windows. T represents the number of tokens in each chunk.
# If T = 8, it means that each sequence consists of 8 tokens.

# C: Channel size, the dimensionality of the input space.
# Each token in the sequence is represented as a one-hot vector of size C, 
# where C is the number of unique tokens in the vocabulary.
# If C = 65, it means that there are 65 unique tokens in the vocabulary.

    def forward(self, idx, targets=None):

        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) which is batch, time, channel 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    

    # The job of this function is to generate the next token given the existing context. If I gave you the letters "Th", this function might generate the letter "e" based on the probabilities.
    # We do this by taking the current idx (tensor that represents a series of letters, or tokens), which is a (Batch, Time) dimensioned array of indices. We perform our operations
    # on it, and at the end we sample a new token from our probability distribution. That new token then gets appended to the end of the idx array so that it becomes (Batch, Time + 1) dim.
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [29]:
#Ok now is the time to train the model. Let's make an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [41]:
batch_size = 32
for steps in range(10000):

    #sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.397209882736206


In [42]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



Se n y yorick, il,
Go n manees. wiltheralldipat
Tournghem, ads, so.
CHent
Incy n tim.

HIn sr ulerueis,
SSare haverur s theno Has, est? ce d he bed u.
Loncanororese CAROFLBand
And beerendsthal t ith ane thorere CENSmalar thupat; thelloy
I;
Twing igemiburopin:
COFooweel l d thiy itch pe jefore r a f


In [43]:
# We will now try to implement an attention block. There are some rules we want to lay first:
# 1. Each token only interacts with tokens that have come before it. They cannot look ahead (at future time steps).

# Consider the following example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [44]:
# In this first version of attention, we will use the mean of the previous tokens as the context for the next token.
# This is a very simple form of attention, but it will help us to understand the concept.
# This first version is crude and just uses a couple for loops.

#Version 1:
xbow = torch.zeros((B,T,C)) # x bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)


In [45]:
# Here is another way of doing it that uses a 'mathematical trick' to make it faster.
# This uses matrix multiplication to compute the bag of words.
# the lower triangular matrix wei with normalized rows performs the same operation as the 
# nested loop that computes the cumulative average. This trick allows you to replace the explicit loops with a single matrix multiplication =))
# allclose might come out to a 0, but the error is super small.

#Version 2:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)
diff = torch.abs(xbow - xbow2)
print(diff.max())  

tensor(3.2363e-08)


In [46]:
# Version 3:

# This version produces the same result but uses softmax.
# Tril is a triangular matrix with all ones at and below the diagonal
# Wei gets set to 0s, but then the positions where tril is 0, wei gets them placed with -inf
# When you softmax this, the 0s go to 1 and the -inf go to 0, then the row gets normalized, and out pops the same matrix as above!

# Long story short, you can do weighted aggregations of past elements by using matrix multiplication of lower triangular entries. The entries
# themselves represent how much each element 'fuses' or 'affects' your current position

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # The -inf is equivalent to saying 'tokens in the future cannot interfere with past/present'
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)
diff = torch.abs(xbow - xbow3)
print(diff.max())  



tensor(3.2363e-08)


In [47]:
# Lets get to the self-attention part. In our language model, we want to have the historical context of the letters that came before us,
# but the context that is 'important' to us is dependent on what the previous characters actually were. For example, if the last letter
# generated was a 'q', thats super interesting to know because know I'm basically gauranteed to generate a 'u'. But if the last letter was
# a 'y', now I'm not so sure I want a 'u'. Maybe an 'o' makes more sense. Another way of putting this: our weighted affinities (wei matrix)
# shouldn't be uniform. This is an important distinction because up until now, our models had no concept of 'token identity' or 'what is
# this token looking for?' It has always been a crude pattern recognition optimized from backpropagation. Suddenly, now we have a way to 
# say 'these two tokens are really good together according to our target, do more of that!' 

# Version 4: Self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# Here is a single head performing self-attention. By the way self-attention is a great name =) Pay more attention to yourself =)
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) 
wei = F.softmax(wei, dim=1)
v = value(x)
out = wei @ v

In [48]:
# Some notes on attention
# 1. At its core, attention is a communication mechanism. It provides a way for nodes in a graph to look at each other and interact
# more or less depending on their affinities (rated by their dot product of key and query)

# 2. There is no notion of space in attention. It's simply a mathematical act over some vectors. To add this functionality, we need to 
# positionally encode tokens to say 'hey, you are token number 5.'

# 3. Sequences that are in different batches DO NOT talk to eachother. They are processed independently.

# 4. The idea that 'next token generation cannot have inputs from future tokens' is not always true. There are some cases where you want
# all the tokens to communicate, such as sentiment analysis. For this, just delete the masked_fill line.

# 5. Self-attention is when your nodes all have their own values, keys, and queries, and they all talk to eachother. Attention is more 
# general than that. A different example is cross-attention, where your nodes have values, but the keys and queries come from a different set.

# 6. We need to divide our wei by 1 over square root of head_size. If you don't, the variance of wei will be on the order of head_size,
# which means that softmax will be very skewed towards the extremes. By including this factor, the var stays around 1, making softmax
# work much better.

In [53]:
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)


    def __call__(self, x):
        #calculate forward pass
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance. This is the heart of batch norm.
        self.out = self.gamma * xhat + self.beta # add the parameters



    def parameters(self):
        return [self.gamma, self.beta]
    

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32,100)
x = module(x)
x.shape


torch.Size([32, 100])

In [54]:
x[:,0].mean(), x[:,0].std()


(tensor(0.1469), tensor(0.8803))

In [55]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))